# 추론 기반 기법과 신경망

* 통계기반 기법: 학습데이터를 1번에 처리, 어휘 수가 늘수록 계산량 크게 증가
* 추론기반 기법: 미니배치의 학습 샘플씩 반복학습, 순차적으로 신경망 학습하여 가중치 갱신

In [1]:
# 원핫 인코딩: 단어를 고정길이의 벡터로 변환해야 함
# 사실 W 행렬의 첫번째 행벡터를 뽑아낸 것. -> 4장에서 개선 가능
import numpy as np
c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
h = np.matmul(c, W)
print(h)

[[-1.27343452  1.91535692 -1.83792457]]


In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[ 1.67002577 -2.16468086 -0.14927095]]


# CBOW 모델

**CBOW (continuous bag of words) 모델**

* 주변 단어들로부터 타깃 단어를 추측하기
* (복수의 입력층) -> (동일 가중치행렬) -> (결과값 평균, 은닉층) -> (가중치행렬) -> (출력층)
* 출력층의 뉴런은 각 단어에 대응, softmax 적용해서 확률 얻기
* 입력층과 은닉층 사이 가중치 행렬을 임베딩 행렬로 사용
* 학습: 소프트맥스 및 교차 엔트로피 오차 이용

`cbow_predict.py`를 확인하세요.


# 학습 데이터 준비

In [2]:
# 1. 전처리 및 id부여
import sys
sys.path.append('..')
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word2id, id2word = preprocess(text)
print(corpus)
print(id2word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [3]:
# 2. 맥락과 타깃 만들기
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size] # 첫, 끝 window_size개의 토큰은 포함하지 않음
    contexts = []
    
    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx+t])
        contexts.append(cs)
    return np.array(contexts), np.array(target)

In [4]:
contexts, target = create_contexts_target(corpus, window_size=1)
print(contexts)
print(target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
[1 2 3 4 1 5]


In [6]:
# 3. 원핫 표현으로 변환
# 기본적으로 차원이 1 증가한다.
def convert_one_hot(corpus, vocab_size):
    N = corpus.shape[0]
    if corpus.ndim == 1:
        # 단어 id 목록이 1차원일 때
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1
    elif corpus.ndim == 2:
        # 단어 id 목록이 2차원일 때, 즉 sequence가 여러개일 때
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for i, seq in enumerate(corpus):
            for j, word_id in enumerate(seq):
                one_hot[i, j, word_id] = 1
    return one_hot

In [7]:
import sys
sys.path.append('..')
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word2id, id2word = preprocess(text)
contexts, target = create_contexts_target(corpus, window_size=1)
vocab_size = len(word2id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

In [8]:
print(contexts) # (6, 2, 7)
print(target) # (6, 7)

[[[1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0]]

 [[0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0]]

 [[0 0 0 1 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1]]]
[[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]]


이후 모형 구현은 `simple_cbow.py`, `train.py`를 보세요.

# skip-gram 모델

* 중앙의 단어로부터 주변의 여러 단어를 예측 (타깃으로부터 맥락을 예측)
* 개별 손실을 더한 값이 최종 손실
* 단어 분산 표현의 정밀도 면에서 CBOW보다 더 좋은 결과를 보임
* 단, 학습 속도는 CBOW가 빠름 (skip-gram은 손실을 맥락 수만큼 구해야 함)

구현은 `simple_skip_gram.py`를 보세요.